In [4]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds
import numpy as np
from rasterio.mask import mask
from shapely.geometry import mapping
import geopandas as gpd

In [97]:
# Rock Island, Moline, Davenport, Bettendorf

In [98]:
# Quad cities bounds

min_lon = -90.6879323
min_lat = 41.4150156
max_lon = -90.4252158
max_lat = 41.6242105

In [100]:
def reproject_raster(input_path, reproj_path, dst_crs):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        kwargs = src.meta.copy()
        kwargs.update({
            "crs": dst_crs,
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(reproj_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )

    print("Reprojected to 4326:", reproj_path)

In [5]:
def plot_raster(output_path, title):
    with rasterio.open(output_path) as src:
        data = src.read(1)

        # Extract bounds for proper geographic axes
        bounds = src.bounds
        extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)

        plt.figure(figsize=(10, 8))

        # Show raster with a geographic extent
        img = plt.imshow(
            data,
            cmap="viridis",
            extent=extent,
            origin="upper"   # important for north-up orientation
        )

        plt.title(title)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")

        # Add colorbar
        cbar = plt.colorbar(img)
        cbar.set_label("Value")

        plt.tight_layout()
        plt.show()

In [102]:
def crop_raster_to_boundaries(reproj_path, cropped_path, gdf):

    with rasterio.open(reproj_path) as src:
        print("Raster CRS:", src.crs)
        print("Raster bounds before crop:", src.bounds)

        # Reproject city polygons to match raster CRS if needed
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)

        # Get list of geojson-like geometry dicts
        geoms = [mapping(geom) for geom in gdf.geometry]

        # Mask & crop
        out_image, out_transform = mask(
            src,
            geoms,
            crop=True,   # crop to the extent of the geometries
            nodata=src.nodata
        )

        # Update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

    # Write the cropped raster
    with rasterio.open(cropped_path, "w", **out_meta) as dst:
        dst.write(out_image)

    print("Saved cropped raster to:", cropped_path)

    # Optional sanity check
    with rasterio.open(cropped_path) as r:
        print("Cropped bounds:", r.bounds)
        print("Cropped shape:", r.width, "x", r.height)

In [103]:
def crop_raster_to_bbox(reproj_path, cropped_path, min_lon, min_lat, max_lon, max_lat):
    with rasterio.open(reproj_path) as src:
        # Make sure we're really in 4326
        print("CRS:", src.crs)
        print("Bounds before crop:", src.bounds)

        # Build a window from the bbox in 4326
        window = from_bounds(
            min_lon, min_lat, max_lon, max_lat,
            transform=src.transform
        )

        # Get the new transform for this window
        window_transform = src.window_transform(window)

        # Prepare metadata for cropped raster
        kwargs = src.meta.copy()
        kwargs.update({
            "height": int(window.height),
            "width": int(window.width),
            "transform": window_transform
        })

        with rasterio.open(cropped_path, "w", **kwargs) as dst:
            dst.write(src.read(window=window))

    print("Saved cropped quad-cities raster to:", cropped_path)

    # Optional sanity check
    with rasterio.open(cropped_path) as r:
        print("Cropped bounds:", r.bounds)
        print("Cropped shape:", r.width, "x", r.height)

In [104]:
def clean_NBS_raster(input_path, output_path):
    allowed = {21, 31, 43, 52, 71, 81, 90, 95}

    with rasterio.open(input_path) as src:
        data = src.read(1)  # single-band raster
        profile = src.profile

    # Create a mask for allowed values
    mask_allowed = np.isin(data, list(allowed))

    # Create cleaned array
    cleaned = np.where(mask_allowed, data, 0).astype(profile["dtype"])

    # Save cleaned raster
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(cleaned, 1)

    print("Saved cleaned raster:", output_path)

In [105]:
def resample_NBS_raster_to_flood_grid(
    nbs_raster_path,  # e.g. "./models/flooding/NBS_others_5m_4326_cropped.tif"
    flood_ref_path,   # e.g. "./models/flooding/2020_2040_NbS_4326_cropped.tif"
    output_path       # e.g. "./models/flooding/NBS_others_5m_4326_cropped_resampled.tif"
):
    with rasterio.open(nbs_raster_path) as src_mask, \
         rasterio.open(flood_ref_path) as src_ref:

        # Take target grid from flood raster
        dst_height, dst_width = src_ref.height, src_ref.width

        print("Source (mask) shape:", src_mask.height, src_mask.width)
        print("Target (flood) shape:", dst_height, dst_width)

        # Decide nodata for the uint8 mask
        # Use existing mask nodata if valid, otherwise 0
        mask_dtype = src_mask.dtypes[0]
        nodata = src_mask.nodata

        if nodata is None:
            nodata = 0
        else:
            # Ensure nodata is within dtype range
            info = np.iinfo(mask_dtype)
            if nodata < info.min or nodata > info.max:
                nodata = 0

        # Prepare destination array filled with nodata
        dst_data = np.full((dst_height, dst_width), nodata, dtype=mask_dtype)

        reproject(
            source=rasterio.band(src_mask, 1),
            destination=dst_data,
            src_transform=src_mask.transform,
            src_crs=src_mask.crs,
            dst_transform=src_ref.transform,
            dst_crs=src_ref.crs,
            src_nodata=src_mask.nodata,
            dst_nodata=nodata,
            resampling=Resampling.nearest,   # categorical
        )

        # Build output profile: grid like flood, dtype/nodata like mask
        dst_profile = src_ref.profile.copy()
        dst_profile.update(
            height=dst_height,
            width=dst_width,
            dtype=mask_dtype,
            nodata=nodata,
            count=1,
        )

        with rasterio.open(output_path, "w", **dst_profile) as dst:
            dst.write(dst_data, 1)

    print("Resampled NBS mask written to:", output_path)

##### Run the cells below to prepare flooding data

In [ ]:
files = ['2020_2040_NbS', '2020_2040_noNbS', '2050_2080_NbS', '2050_2080_noNbS', '2080_2100_NbS', '2080_2100_noNbS', 'NbS_others_5m']

for file in files:
    input_path = f"./models/flooding_/{file}.tif"
    reproj_path = f"./models/flooding_/{file}_4326.tif"

    dst_crs = "EPSG:4326"
    reproject_raster(input_path, reproj_path, dst_crs)

    cropped_path = f"./models/flooding/{file}_4326_cropped.tif"
    crop_raster_to_bbox(reproj_path, cropped_path, min_lon, min_lat, max_lon, max_lat)

In [ ]:
nbs_cropped_4326 = "./models/flooding/NbS_others_5m_4326_cropped.tif"
nbs_cropped_4326_cleaned = "./models/flooding/NBS_others_5m_4326_cropped_cleaned.tif"
nbs_cropped_4326_cleaned_resampled = "./models/flooding/NBS_others_5m_4326_cropped_cleaned_resampled.tif"
flood_ref_path = "./models/flooding/2020_2040_NbS_4326_cropped.tif"

clean_NBS_raster(nbs_cropped_4326, nbs_cropped_4326_cleaned)
resample_NBS_raster_to_flood_grid(
    nbs_cropped_4326_cleaned,
    flood_ref_path,
    nbs_cropped_4326_cleaned_resampled
)

Saved cleaned raster: ./models/flooding/NBS_others_5m_4326_cropped_cleaned.tif
Source (mask) shape: 12839 16123
Target (flood) shape: 2064 2592
Resampled NBS mask written to: ./models/flooding/NBS_others_5m_4326_cropped_cleaned_resampled.tif


In [ ]:
from backend.models.flooding.scripts.flood_simulation import simulate_flood_projection


simulate_flood_projection(
    (-90.6879323, 41.4150156, -90.60, 41.525),
    "B",
    year="2020 - 2040",
    use_NBS_classes= [
        # "Bioswales/Infiltration trenches",
        # "Permeable pavements",
        # "Retention ponds",
        # "Infiltration trench",
        "Bioswales",
        "Constructed wetlands",
    ],
)

Combined flood raster shape: (1085, 868)
Saved combined flood raster to: ./data/served/raster/B.tif
